In [1]:
import sys
sys.path.append('../../')

# BERT_FINE_TUNE_PATH = '../../../bert_fine_tune/'
# sys.path.append(BERT_FINE_TUNE_PATH)

In [2]:
# from textpair.preprocess.dummy_preprocessor import DummyPreprocessor
# from textpair.analyze.bert_analyzer import BertAnalyzer
# from textpair.vectorize.bert_vectorizer import BertVectorizer
# from textpair.model.bert_model import BertModel
# from textpair.pair_ann import PairAnn
# from textpair.semantic.base_semantic import BaseSemantic

import logging

logging.basicConfig(level=logging.INFO)

import torch
from pytorch_pretrained_bert.modeling_fine_tune import BertForPairWiseClassification
from pytorch_pretrained_bert import BertTokenizer

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_pretrained_bert.modeling_fine_tune:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [3]:
FINE_TUNED_PATH = '/efs/fine_tune/lcqmc/pairwise/lcqmc_fine_tune_40_1_3_5e-6/'

In [4]:
model = BertForPairWiseClassification.from_pretrained(FINE_TUNED_PATH)
tokenizer = BertTokenizer.from_pretrained(FINE_TUNED_PATH)

INFO:pytorch_pretrained_bert.modeling_fine_tune:loading weights file /efs/fine_tune/lcqmc/pairwise/lcqmc_fine_tune_40_1_3_5e-6/pytorch_model.bin
INFO:pytorch_pretrained_bert.modeling_fine_tune:loading configuration file /efs/fine_tune/lcqmc/pairwise/lcqmc_fine_tune_40_1_3_5e-6/config.json
INFO:pytorch_pretrained_bert.modeling_fine_tune:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file /efs/fine_tune/lcqmc/pairwise/lcqmc_fine_tune_40_1_

In [5]:
def bert_sim(text1, text2):
    tokens1 = ['[CLS]'] + tokenizer.tokenize(text1) + ['[SEP]']
    tokens2 = ['[CLS]'] + tokenizer.tokenize(text2) + ['[SEP]']
    ids1 = tokenizer.convert_tokens_to_ids(tokens1)
    ids2 = tokenizer.convert_tokens_to_ids(tokens2)
    segs1 = [0] * len(ids1)
    segs2 = [0] * len(ids2)
    tokens_tensor1 = torch.tensor([ids1])
    segments_tensor1 = torch.tensor([segs1])
    tokens_tensor2 = torch.tensor([ids2])
    segments_tensor2 = torch.tensor([segs2])
    model.eval()
    with torch.no_grad():
        cos_sim, pos_prob, vec1, vec2 = model(tokens_tensor1, tokens_tensor2, segments_tensor1, segments_tensor2)
    return cos_sim[0].item(),pos_prob[0].item(), vec1, vec2

# 一般测试

In [6]:
text1 = "我很高兴"
text2 = "我很开心"
cos_sim,pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9461959004402161, 0.9730979204177856)

In [7]:
text1 = "我很开心"
text2 = "我很高兴"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9461959004402161, 0.9730979204177856)

In [8]:
text1 = "我很高兴"
text2 = "我特别特别开心"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.928837776184082, 0.964418888092041)

In [9]:
text1 = "我很高兴"
text2 = "我其实觉得自己很开心"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.882889986038208, 0.941444993019104)

In [10]:
text1 = "我特别特别开心"
text2 = "我其实觉得自己很开心"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9343717098236084, 0.9671858549118042)

In [11]:
text1 = "我很高兴"
text2 = "我不开心"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.19332632422447205, 0.4033368229866028)

In [12]:
text1 = "我很高兴"
text2 = "我不高兴"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.1439635306596756, 0.4280182421207428)

In [13]:
text1 = "我很高兴"
text2 = "我很高兴"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9998999834060669, 0.9999499917030334)

In [14]:
text1 = "我很伤心"
text2 = "我很难过"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9520829916000366, 0.9760414958000183)

In [15]:
text1 = "真好"
text2 = "不错"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.8418208956718445, 0.9209104776382446)

In [16]:
text1 = "高兴"
text2 = "开心"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.7031331062316895, 0.8515665531158447)

In [17]:
text1 = "大家好才是真的好"
text2 = "大家好才是真的好"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9998999834060669, 0.9999499917030334)

In [18]:
text1 = "为什么能开出腾讯信用却没有微粒贷朋友的没用腾讯信用却有30000的额度呢"
text2 = "我钱包里没有你们这个应用"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.3624228239059448, 0.3187885880470276)

In [19]:
text1 = "我也不知道"
text2 = "好吧"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.401532918214798, 0.7007664442062378)

In [20]:
text1 = "深度学习"
text2 = "机器学习"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6687267422676086, 0.8343633413314819)

In [21]:
text1 = "机器学习"
text2 = "深度学习"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6687267422676086, 0.8343633413314819)

In [22]:
text1 = "人民团体是什么"
text2 = "人民团体是指"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9628378748893738, 0.9814189672470093)

# 百度-车牌，不太一致

In [23]:
text1 = "车头如何放置车牌"
text2 = "前牌照怎么装"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.08089974522590637, 0.459550142288208)

In [24]:
text1 = "车头如何放置车牌"
text2 = "如何办理北京车牌"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.7838859558105469, 0.8919429779052734)

In [25]:
text1 = "车头如何放置车牌"
text2 = "后牌照怎么装"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.12562254071235657, 0.4371887445449829)

# 百度-信号

In [26]:
text1 = "信号忽强忽弱"
text2 = "信号忽高忽低"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6460959315299988, 0.8230479955673218)

In [27]:
text1 = "信号忽强忽弱"
text2 = "信号忽左忽右"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.2920767664909363, 0.6460384130477905)

In [28]:
text1 = "信号忽强忽弱"
text2 = "信号忽然中断"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.3235416114330292, 0.6617708206176758)

# 百度-机器学习

In [29]:
text1 = "如何学好深度学习"
text2 = "深入学习习近平讲话材料"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.04421834275126457, 0.47789081931114197)

In [30]:
text1 = "如何学好深度学习"
text2 = "机器学习教程"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6169852018356323, 0.8084926009178162)

In [31]:
text1 = "如何学好深度学习"
text2 = "人工智能教程"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.5044037103652954, 0.7522018551826477)

# 百度-香蕉的翻译，偏小但排序一致

In [32]:
text1 = "香蕉的翻译"
text2 = "香蕉用英文怎么说"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9225574731826782, 0.9612787365913391)

In [33]:
text1 = "香蕉的翻译"
text2 = "香蕉怎么吃"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6751666069030762, 0.8375833034515381)

In [34]:
text1 = "香蕉的翻译"
text2 = "桔子用英文怎么说"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6437533497810364, 0.8218766450881958)

# 百度-腹泻，排序有差别

In [35]:
text1 = "小儿腹泻偏方"
text2 = "宝宝拉肚子偏方"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.699819803237915, 0.8499099016189575)

In [36]:
text1 = "小儿腹泻偏方"
text2 = "小儿感冒偏方"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.5160297751426697, 0.7580149173736572)

In [37]:
text1 = "小儿腹泻偏方"
text2 = "腹泻偏方"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.9587470889091492, 0.979373574256897)

# 百度-LOL，数值偏小，但排序一致

In [38]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "lol攻略"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6479702591896057, 0.8239850997924805)

In [39]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "英雄联盟服务器升级"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.6615750193595886, 0.8307875394821167)

In [40]:
text1 = "英雄联盟好玩吗，怎么升级"
text2 = "怎么打好lol"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.7770408391952515, 0.8885204195976257)

# 百度-红米

In [41]:
text1 = "红米更新出错"
text2 = "红米升级系统出错"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.841148853302002, 0.920574426651001)

In [42]:
text1 = "红米更新出错"
text2 = "红米账户出错"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.7717515826225281, 0.8858758211135864)

In [43]:
text1 = "红米更新出错"
text2 = "如何买到小米手机"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.5025741457939148, 0.2487129271030426)

# 百度-李彦宏

In [44]:
text1 = "李彦宏是百度公司创始人"
text2 = "百度是李彦宏创办的"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.905514657497406, 0.9527573585510254)

In [45]:
text1 = "李彦宏是百度公司创始人"
text2 = "马化腾创办了腾讯公司"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.7203862071037292, 0.860193133354187)

In [46]:
text1 = "李彦宏是百度公司创始人"
text2 = "姚明是NBA的著名球星"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.5420638918876648, 0.7710319757461548)

# 百度-中国历史

In [47]:
text1 = "中国有五千年的历史"
text2 = "中国是个历史悠久的国家"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.4960595667362213, 0.7480297684669495)

In [48]:
text1 = "中国有五千年的历史"
text2 = "中国有很多少数民族"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.1780521124601364, 0.5890260338783264)

In [49]:
text1 = "中国有五千年的历史"
text2 = "中国有13亿人口"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.05039781332015991, 0.47480109333992004)

# 百度-北京奥运会，偏小，但数值一致

In [50]:
text1 = "北京成功申办了2008年奥运会"
text2 = "2008年奥运会在北京举行"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.5722275376319885, 0.7861137390136719)

In [51]:
text1 = "北京成功申办了2008年奥运会"
text2 = "伦敦奥运会在2012年举行"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(0.20796354115009308, 0.6039817929267883)

In [52]:
text1 = "北京成功申办了2008年奥运会"
text2 = "东京奥运会即将举办"
cos_sim, pos_prob, vec1, vec2 = bert_sim(text1, text2)
cos_sim, pos_prob

(-0.02775203064084053, 0.4861239790916443)